#### Instantiate the Python SDK client, Setup variables.

In [5]:
import os
import sys
import utilities
from IPython.display import Image, display
from eecloud.cloudsdk import CloudSDK, SDKBase
from eecloud.models import *

root_path = os.environ.get("HOME")
jupyter_user = os.environ.get("JUPYTERHUB_USER")
project_root = local_path = os.path.join(root_path, "JupyterSamples", "PreAndPostTasks")

local_path = os.path.join(project_root, "Scripts") 
datahub_path = f"JupyterSamples/{jupyter_user}"
simulation_body = os.path.join(local_path, "simulation_body.json")

solution_download_directory = os.path.join(project_root, "Downloads")
cli_path = os.environ.get("cloud_cli_path")
environment: str = "Aquila"

if environment is None:
    raise Exception("environment value must be configured")

print(f"Python Path: {sys.executable}")
pxc = CloudSDK(cli_path="/srv/cli/plexos-cloud") # I will spend more time setting environment variables on JupyterHub to make this simpler

Python Path: /opt/tljh/user/bin/python
Selected CLI: /srv/cli/plexos-cloud


#### Set Environment

In [6]:
try:
    environment_response = pxc.environment.set_user_environment(environment, print_message=False)
    environment_data = pxc.environment.get_final_response(environment_response)
    print(f"{environment_data.EventData.Environment} selected, please authenticate")
except Exception as ex:
    print(ex)

Aquila selected, please authenticate


#### Login to PLEXOS Cloud using SSO - This will only work locally not on Hub

In [3]:
try:
    login_response = pxc.auth.login(print_message=False)
    login_data = pxc.auth.get_final_response(login_response)
    print(f"{login_data.EventData.UserName} ({login_data.EventData.TenantName}) logged into {login_data.EventData.Environment}")
except Exception as ex:
    print(ex)

Anthony Griesel (EE Tenant) logged into Aquila


#### Login to PLEXOS Cloud using Client credentials

In [7]:
ee_tenant_id = 'd469f645-ff96-4d9d-9879-09cb1f87b3d7'
ee_client_id = '0ce43afb-e94a-4ef5-bb06-ce70b8d71361'
ee_client_secret = 'DXIFZHAdnRaKi54Tq/psnyWEbToUq/Yn6A3Uf/WLV0g='

try:
    command_responses = pxc.auth.login_client_credentials(tenant_id=ee_tenant_id, client_id=ee_client_id, client_secret=ee_client_secret, use_client_credentials=True, print_message=False)
    last_command_response: CommandResponse[Contracts_LoginResponse] = pxc.auth.get_final_response(command_responses)
    if last_command_response is not None and last_command_response.Status == "Success":
        data: Contracts_LoginResponse = last_command_response.EventData
        print(f"{data.TenantName} logged into {data.Environment}")
    else:
        print(f"Failure to login: {last_command_response.Message}")
except Exception as ex:
    print(ex)

EE Tenant logged into Aquila


#### Setup Datahub to local folder sync

In [4]:
try:
    print(f"Attempting to map local path: {local_path} to {datahub_path} on Datahub")
    command_responses = pxc.datahub.map_folder(local_path, datahub_path, print_message=False)
    last_command_response: CommandResponse[Contracts_DatahubMapResponse] = pxc.datahub.get_final_response(command_responses)
    if last_command_response is not None and last_command_response.Status == "Success":
        data: Contracts_DatahubMapResponse = last_command_response.EventData
        print(f"{data.LocalPath} mapped to {data.RemotePath} on Datahub")
    elif last_command_response is not None:
        print(last_command_response.Message)
except Exception as ex:
    print(ex)

Attempting to map local path: /home/jupyter-anthony.griesel/JupyterSamples/PreAndPostTasks/Scripts to JupyterSamples/anthony.griesel on Datahub
/home/jupyter-anthony.griesel/JupyterSamples/PreAndPostTasks/Scripts mapped to JupyterSamples/anthony.griesel on Datahub


#### Sync data in between mapped local folder and Datahub

In [8]:
try:
    command_responses = pxc.datahub.sync(local_path_to_sync=local_path, verify_downloads=False, replace_local_files_on_conflict=True, print_message=False)
    last_command_response: CommandResponse[Contracts_DatahubCommandResponse] = pxc.datahub.get_final_response(command_responses)
    if last_command_response is not None and last_command_response.Status == "Success":
        data: Contracts_DatahubCommandResponse = last_command_response.EventData
    
        if len(data.DatahubResourceResults) == 0:
            print("No changes found")
    
        for item in data.DatahubResourceResults:
            if item.Success == True:
                print(item.RelativeFilePath)
            elif "Skipping upload" in item.FailureReason:
                print(f"Skipped: {item.RelativeFilePath} - Identical")
            else:
                print(f"Failed {item.RelativeFilePath} - {item.FailureReason}")
    else:
        print("Failed to sync data between local and remote filesystems")
except Exception as ex:
    print(ex)

Failed JupyterSamples/anthony.griesel/.ipynb_checkpoints/requirements-checkpoint.txt - Status - BadRequest,Bad Request,{"type":"https://portal.energyexemplar.com/unified-help/plexos-cloud/#t=Troubleshooting%2FCloud_Errors.htm","title":"GeneralValidationError","detail":"Invalid or missing param.","errorCode":"CE40003","additionalMessages":["The session 0ebd2924-2b7c-4c0f-870f-5a85df207307 is invalid"],"traceId":"0HNB75UTFGVNF:0000001B"}
Failed JupyterSamples/anthony.griesel/.ipynb_checkpoints/query_write_memberships-checkpoint.py - Status - BadRequest,Bad Request,{"type":"https://portal.energyexemplar.com/unified-help/plexos-cloud/#t=Troubleshooting%2FCloud_Errors.htm","title":"GeneralValidationError","detail":"Invalid or missing param.","errorCode":"CE40003","additionalMessages":["The session c8651596-a69d-426c-a239-500d402d3afc is invalid"],"traceId":"0HNB75UTFGVNF:00000021"}


#### List Datahub files

In [5]:
try:
    search_response: list[CommandResponse[Contracts_DatahubSearchResponse]] = pxc.datahub.search([f"{datahub_path}/**"], print_message=False)
    search_data: Contracts_DatahubSearchResponse = pxc.datahub.get_final_response(search_response)

    for item in search_data.EventData.DatahubSearchResults:
        if item.IsDeleted == False:
            print(f"{item.RelativePath} - Total Versions: {len(item.Versions)} : Latest Version: {item.LatestServerVersion}")
except Exception as ex:
    print(ex)

JupyterSamples/anthony.griesel/configure_duck.py - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/directorymapping_example.json - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/query_write_memberships.py - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/requirements.txt - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/clean_workflow_files.py - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/manage_datahub_files.py - Total Versions: 1 : Latest Version: 0
JupyterSamples/anthony.griesel/simulation_body.json - Total Versions: 4 : Latest Version: 3
JupyterSamples/anthony.griesel/query_data.py - Total Versions: 2 : Latest Version: 1
JupyterSamples/anthony.griesel/.ipynb_checkpoints/simulation_body-checkpoint.json - Total Versions: 3 : Latest Version: 2
JupyterSamples/anthony.griesel/.ipynb_checkpoints/query_data-checkpoint.py - Total Versions: 1 : Latest Version: 0


#### Define Tasks in Simulation Payload

Manually update the [simulation_body.json](Scripts/simulation_body.json) located in the Scripts directory. The following must be updated
- StudyId
- ChangesetId
- Models
- SimulationData
- SimulationEngine
- The tasks under SimulationTasks must be updated to include your Datahub path names. Documentation here: [Task Documentation](https://portal.energyexemplar.com/unified-help/plexos-cloud/#t=Simulations%2FPre-_and_Post-Simulation_Tasks.htm&rhsearch=pre%20and%20post&ux=search) 
  - TaskType: - Pre or Post
  - Files: Datahub Path and Version
  - Arguments: If python script needs to be executed, the arguments must contain "python3 yourscript.py"

*** If new scripts are added, the sync process must be run again. 


```json
...
{
    "Name": "TASK NAME",
    "TaskType": "Pre",
    "Files": [
        {
            "Path": "JupyterSamples/<user.name>/query_write_memberships.py",
            "Version": null
        },
        {
            "Path": "JupyterSamples/<user.name>/requirements.txt",
            "Version": 1
        }
    ],
    "Arguments": "python3 query_write_memberships.py",
    "ContinueOnError": true,
    "ExecutionOrder": 1
}
...

```

#### Enqueue Simulation, Wait for Completion. Download Results

In [17]:
try:
    enqueue_response = pxc.simulation.enqueue_simulation(simulation_body, print_message=False)
    enqueue_data : Contracts_EnqueueSimulationResponse = pxc.simulation.get_final_response(enqueue_response)
    simulation = enqueue_data.EventData.SimulationStarted[0]

    simulation_id = simulation.Id.Value
    execution_id = simulation.ExecutionId.Value

    print(f"Simulation: {simulation_id} enqueued")

    utilities.wait_simulation_finish(pxc, simulation_id)
    simulation_result = utilities.get_simulation(pxc, simulation_id="c17344a5-21a1-4122-a25f-1e44efa49839")

    #optionally if multiple simulations were enqueued we can wait for all executions to finish 
    #execution_result = utilities.get_executions(pxc, execution_id=simulation_result.ExecutionId.Value)
    #execution_result = utilities.wait_execution_finish(pxc, execution_id=simulation_result.ExecutionId.Value)

    if simulation_result.Status in ['CompletedSuccess']:
        solution_id = simulation_result.ModelIdentifiers[0].Id
        print(f"Simulation complete. {simulation_result.Status} Downloading artifacts for solution: {solution_id}")       
        utilities.download_solution_data(pxc, solution_id , solution_download_directory)
        print(f"Solution data downloaded to {solution_download_directory}")
    else:
        print(f"Simulation finished: {simulation_result.Status} - Possible failure")
except Exception as ex:
    print(ex)
    

Simulation: 412fdad8-d55b-4289-9f3d-9880be256d70 enqueued

Queued...
PreProcessing......
Running...
Postprocessing...
CompletedSuccessDone: CompletedSuccess
Simulation complete. CompletedSuccess Downloading artifacts for solution: c2ecd76f-07c6-4bfc-90a7-57a1ae87d5e4
Contracts_Simulation(SimulationType='None', Id=GuidValue(Value='c17344a5-21a1-4122-a25f-1e44efa49839'), ExecutionId=GuidValue(Value='0eedbf18-9eb0-49b4-9b78-d6a0e6b0c93f'), StudyId=GuidValue(Value='88b28ec1-2036-4fde-bad0-180406be58b5'), ChangeSetId=GuidValue(Value='407b542e-0867-48ef-af55-44298b1aa6f5'), CreatedByUser=Contracts_User(Name=None, EmailAddress=None, Id='0ce43afb-e94a-4ef5-bb06-ce70b8d71361'), Source='Jupyter', RequestedCpuCores=2, MinimumMemoryInGb=16.0, CreatedAt='2025-03-19T23:57:13.4750037+00:00', LastUpdatedAt='2025-03-19T23:59:18.0024721+00:00', Models=['No Carbon Price or Flow Constraint'], Status='CompletedSuccess', ModelIdentifiers=[Contracts_ModelIdentifier(Id='c2ecd76f-07c6-4bfc-90a7-57a1ae87d5e4', 